# Lightgbm optmizado
Entrenamiento LightGBM

In [3]:
# 1) Montar Drive e instalar LightGBM
# from google.colab import drive
# drive.mount('/content/drive')
# #!pip install lightgbm --quiet

# 2) Rutas en Drive
import os
BASE_FEATURES = '/Users/indianaabeledo/Documents/Maestria/laboratorio_III/labo3-2025v/entrega_final/features/'
BASE_OUTPUTS  = '/Users/indianaabeledo/Documents/Maestria/laboratorio_III/labo3-2025v/entrega_final/output/'
os.makedirs(BASE_OUTPUTS, exist_ok=True)

FEATURES_PATH = os.path.join(BASE_FEATURES, 'dataset_features_product_id.pkl')
MODEL_PATH    = os.path.join(BASE_OUTPUTS,  'modelo_lgbm.txt')
PRED_PATH     = os.path.join(BASE_OUTPUTS,  'pred_modelo_lightgbm_best.csv')
IMP_PATH      = os.path.join(BASE_FEATURES, 'feature_importance.csv')

In [4]:
# 3) Importar librerías
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import gc

# 4) Carga de features desde CSV
print("📥 Cargando dataset de features…")
df = pd.read_pickle(FEATURES_PATH)
print(f"✅ Dataset: {df.shape[0]} filas, {df.shape[1]} columnas.")

📥 Cargando dataset de features…
✅ Dataset: 28080 filas, 592 columnas.


In [5]:
# 5) Asegurar columna target
if 'target' not in df.columns:
    df = df.sort_values(['product_id','periodo']).reset_index(drop=True)
    df['target'] = df.groupby('product_id')['tn'].shift(-2)

# 6) Filtrar y log-transform
df = df[df['target'].notna()].reset_index(drop=True)
df['target_log'] = np.log1p(df['target'].clip(lower=0))

In [6]:
# 7) Reducir cardinalidad en categóricas
def reducir_card(serie, max_unique=30):
    top = serie.value_counts().nlargest(max_unique).index
    return serie.where(serie.isin(top), other='Other')
for col in ['cat1','cat2','cat3']:
    if col in df.columns:
        df[col] = df[col].astype('category')
        if 'Other' not in df[col].cat.categories:
            df[col] = df[col].cat.add_categories(['Other'])
        df[col] = reducir_card(df[col]).astype('category')

In [7]:
# 8) Definir lista de features dinámicamente
print("🔄 Definiendo lista de features…")
exclude = [
    #'cluster_dtw',
    #'tn_rollmax2',
    #'tn_roll2',
    #'tn_mean_all',
    'share_total',
    'product_id',
    'n_customers',
    'cat2','cat3',
    'period', 'periodo','prod_start',
    'target','target_log','tn',
    'brand','cust_request_qty','cust_request_tn',
    'plan_precios_cuidados','sku_size'
]

features = [
    c for c in df.columns
    if c not in exclude and pd.api.types.is_numeric_dtype(df[c])
]

categ_feats = [c for c in ['cat1',
                           'cluster_dtw'
                           ] if c in features]

print(f"✅ {len(features)} features seleccionadas.")

🔄 Definiendo lista de features…
✅ 577 features seleccionadas.


In [8]:
# 9) Split temporal train/test
max_ord = df['period_ordinal'].max()
train   = df[df['period_ordinal'] <= max_ord - 2]
test    = df[df['period_ordinal'] >  max_ord - 2]
X_train = train[features];  y_train = train['target_log']
X_test  = test[features];   y_test  = test['target_log']
y_real  = test['target']

In [9]:
# 10) Entrenamiento LightGBM

dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=categ_feats)
dvalid = lgb.Dataset(X_test,  label=y_test,  reference=dtrain, categorical_feature=categ_feats)
params = {
    'objective':'regression','metric':'mae','boosting_type':'gbdt',
    'max_depth': 10, 'learning_rate': 0.05663964996081039, 'min_child_weight': 3, 'subsample': 0.8316655287432297,
    'num_leaves': 39, 'feature_fraction': 0.8247124868682788, 'bagging_fraction': 0.9595500390363261, 'bagging_freq': 8,
    'max_bin': 652, 'colsample_bytree': 0.8474007166109918, 'reg_alpha': 0.8217385123006359, 'reg_lambda': 0.9295570679904743, 'n_estimators': 112,'min_child_samples':20,
    'random_state':42,'verbosity':-1
}
print("🚀 Entrenando LightGBM…")
model = lgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    valid_sets=[dtrain, dvalid],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)
    ]
)
print("✅ Entrenamiento completado.")

# 11) Exportar feature importance
imp_gain  = model.feature_importance(importance_type='gain')
imp_split = model.feature_importance(importance_type='split')
names     = model.feature_name()
norm_gain = imp_gain / imp_gain.sum()
df_imp = pd.DataFrame({
    'feature':names,'importance_gain':imp_gain,
    'importance_gain_norm':norm_gain,'importance_split':imp_split
}).sort_values('importance_gain', ascending=False)
df_imp.to_csv(IMP_PATH, index=False)

# 11) Visualizar feature importance
imp_gain  = model.feature_importance(importance_type='gain')
names     = model.feature_name()
norm_gain = imp_gain / imp_gain.sum()

df_imp = pd.DataFrame({
    'feature':              names,
    'importance_gain_norm': norm_gain,
}).sort_values('importance_gain_norm', ascending=False)

# Mostrar top 20 en consola
print("🔍 Top 20 features por importancia normalizada:")
print(df_imp.head(20).to_string(index=False))

# Gráfico de barras horizontal
plt.figure(figsize=(8,6))
top20 = df_imp.head(20)[::-1]  # invertir para gráfico
plt.barh(top20['feature'], top20['importance_gain_norm'])
plt.xlabel('Importancia normalizada')
plt.title('Top 20 Feature Importance')
plt.tight_layout()
plt.show()

# 13) Evaluación en test
pred_log = model.predict(X_test); pred = np.expm1(pred_log)
test['tn_pred'] = pred
mae_val = mean_absolute_error(y_real, pred)
rmse_val = np.sqrt(mean_squared_error(y_real, pred))
print(f"🧪 Test MAE: {mae_val:.3f}, RMSE: {rmse_val:.3f}")

# 14) Predicciones último periodo
df_pred = df[df['period_ordinal']==max_ord]
X_pred  = df_pred[features]

out = pd.DataFrame({'product_id': df_pred['product_id'], 'tn_pred': np.expm1(model.predict(X_pred))})

out.to_csv(PRED_PATH, index=False)

# 14) Guardar modelo
with open(MODEL_PATH, 'w') as f:
    f.write(model.model_to_string())

# 15) Liberar memoria
#gc.collect()

🚀 Entrenando LightGBM…
Training until validation scores don't improve for 50 rounds
[100]	training's l1: 0.183176	valid_1's l1: 0.283786
Did not meet early stopping. Best iteration is:
[91]	training's l1: 0.18788	valid_1's l1: 0.280448
✅ Entrenamiento completado.
🔍 Top 20 features por importancia normalizada:
           feature  importance_gain_norm
       tn_rollmax2              0.392525
          tn_roll2              0.342802
       tn_mean_all              0.104061
          tn_roll4              0.028303
      tn_rollmax11              0.017562
          tn_roll3              0.014357
        tn_max_all              0.011777
          tn_roll5              0.007274
meses_ceros_consec              0.006711
     tn_count_zero              0.005989
       tn_rollmax8              0.005623
         tn_roll11              0.005470
        tn_pct_all              0.005283
       tn_rollmax7              0.003364
       tn_rollmax3              0.003004
     tn_max_last_3              0

🧪 Test MAE: 8.956, RMSE: 27.907


/var/folders/c5/g8wp4j8j1nd226rj0lw9_vz40000gn/T/ipykernel_12805/1280316270.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['tn_pred'] = pred


Mejor LightGBM con un error en kaggle de 0,26

Mejores parámetros: {'max_depth': 10, 'learning_rate': 0.05663964996081039, 'min_child_weight': 3, 'subsample': 0.8316655287432297, 'num_leaves': 39, 'feature_fraction': 0.8247124868682788, 'bagging_fraction': 0.9595500390363261, 'bagging_freq': 8, 'max_bin': 652, 'colsample_bytree': 0.8474007166109918, 'reg_alpha': 0.8217385123006359, 'reg_lambda': 0.9295570679904743, 'n_estimators': 112}
🚀 Entrenando modelo final LightGBM…


    param = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'random_state': 42,
        'verbosity':      -1,
        'min_child_samples':20,
        'max_depth':           trial.suggest_int  ('max_depth',        10, 30),
        'learning_rate':       trial.suggest_float('learning_rate', 0.05, 0.06),
        'min_child_weight':    trial.suggest_int  ('min_child_weight', 3,  5),
        'subsample':           trial.suggest_float('subsample',    0.6,  0.9),
        'num_leaves':       trial.suggest_int  ('num_leaves',       30, 50),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.8, 0.9),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq':     trial.suggest_int  ('bagging_freq',      7, 10),
        'max_bin':          trial.suggest_int  ('max_bin',          500, 800),
        'colsample_bytree':    trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'reg_alpha':           trial.suggest_float('reg_alpha',    0.8,  0.9),
        'reg_lambda':          trial.suggest_float('reg_lambda',   0.0,  1.0),      
        }


    # Número de iteraciones (n_estimators)
    n_estimators = trial.suggest_int('n_estimators', 100,200)

## Errores lightgbm 201912

In [10]:
test.head()

periodo  product_id           tn  cust_request_qty  cust_request_tn  \
32    201909       20001  1660.005615             392.0      1734.241943   
33    201910       20001  1561.505493             367.0      1587.875244   
66    201909       20002  1090.187744             438.0      1095.098267   
67    201910       20002  1979.536377             312.0      2013.363037   
100   201909       20003   967.771179             470.0       977.107788   

     plan_precios_cuidados  n_customers   cat1         cat2      cat3  ...  \
32                       0        166.0     HC  ROPA LAVADO   Liquido  ...   
33                       0        128.0     HC  ROPA LAVADO   Liquido  ...   
66                       0        155.0     HC  ROPA LAVADO   Liquido  ...   
67                       0         89.0     HC  ROPA LAVADO   Liquido  ...   
100                      0        218.0  FOODS     ADEREZOS  Mayonesa  ...   

    es_precios_cuidados    USD_ARS         IPC  holiday_count  cluster_dtw  \
32                    0  58.790001  253.710205              0            5   
33                    0  61.403000  262.066101              2            5   
66                    0  58.790001  253.710205              0            2   
67                    0  61.403000  262.066101              2            2   
100                   0  58.790001  253.710205              0            4   

     prod_start_ord  prod_start_year prod_start_month  target_log      tn_pred  
32              564             2017                1    7.243064  1170.776782  
33              564             2017                1    7.317006  1072.377714  
66              564             2017                1    7.261631  1094.014089  
67              564             2017                1    6.992380  1078.858007  
100             564             2017                1    6.855719   904.910863  

[5 rows x 594 columns]

In [11]:
df_valid = test[test['periodo'] == 201910].copy()

df_valid['abs_error'] = (df_valid['target'] - df_valid['tn_pred']).abs()
df_valid['rel_error'] = (df_valid['abs_error'] / df_valid['target'])
total_error = df_valid['abs_error'].sum()
total_sales = df_valid['target'].sum()
tfe = total_error / total_sales
print(f"\n📉 Total Forecast Error (TFE): {tfe:.4f}")


📉 Total Forecast Error (TFE): 0.2548


In [12]:
# 14. Productos con mayor error
top_errores = df_valid.sort_values('abs_error', ascending=False)
print("\n🔍 Top productos con mayor error absoluto:")
print(top_errores[['product_id', 'target', 'tn_pred', 'abs_error','rel_error']])


🔍 Top productos con mayor error absoluto:
       product_id       target      tn_pred   abs_error  rel_error
33          20001  1504.688599  1072.377714  432.310884   0.287309
407         20012   173.130035   334.821354  161.691319   0.933930
271         20008   195.368546   347.529457  152.160911   0.778840
475         20014   272.028107   390.193327  118.165220   0.434386
781         20023   181.132767   283.990365  102.857599   0.567857
...           ...          ...          ...         ...        ...
22473       20967     1.554080     1.563430    0.009350   0.006017
17033       20674     8.385390     8.377883    0.007507   0.000895
24037       21077     0.215840     0.209910    0.005930   0.027473
16081       20637     4.749640     4.752190    0.002550   0.000537
24819       21140     0.556940     0.556214    0.000726   0.001303

[780 rows x 5 columns]


In [13]:
 #Definir ruta de salida (reusa BASE_OUTPUTS si ya la tienes)
ERROR_CSV_PATH = os.path.join(BASE_OUTPUTS, 'error_201912_lightgbm.csv')

# Exportar a CSV
top_errores.to_csv(ERROR_CSV_PATH, index=False)

print(f"✅ Resultados exportados a: {ERROR_CSV_PATH}")

✅ Resultados exportados a: /Users/indianaabeledo/Documents/Maestria/laboratorio_III/labo3-2025v/entrega_final/output/error_201912_lightgbm.csv
